### Import and Normalisation

We use Z-Score for normalisation. To normalize Date and Time, we calculate nr of days from beginning, and nr of seconds since midnight resulting in two numerical values we can normalize

In [ ]:
import pandas as pd
import numpy as np

# Path to your CSV file
file_path = './Occupancy_Estimation.csv'
df = pd.read_csv(file_path)

df['Date'] = pd.to_datetime(df['Date'])
# Find the earliest date
earliest_date = df['Date'].min()
# Calculate the difference in days from the earliest date
df['Days_Since_Earliest'] = (df['Date'] - earliest_date).dt.days

df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time
# Calculate the number of seconds since the beginning of the day
df['Seconds_Since_Midnight'] = df['Time'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)

df.drop(['Date'], axis=1, inplace=True)
df.drop(['Time'], axis=1, inplace=True)
df.drop(['Room_Occupancy_Count'], axis=1, inplace=True)

columns_to_normalize = df.select_dtypes(include=np.number).columns
df[columns_to_normalize] = (df[columns_to_normalize] - df[columns_to_normalize].mean()) / df[columns_to_normalize].std()

# Print the normalized DataFrame
# print(df)
#df['Days_Since_Earliest']
df


,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Days_Since_Earliest,Seconds_Since_Midnight
0,-1.462961,-1.357708,-1.162277,-1.049633,1.873213,0.118621,0.321083,1.366159,-0.278418,0.262413,-0.237211,-0.363270,-0.354364,0.664436,-0.314733,-0.294014,-0.740905,-0.163279
1,-1.462961,-1.357708,-1.162277,-0.881298,1.873213,0.103763,0.321083,1.366159,2.405433,-0.262910,-0.237211,-0.363270,-0.354364,0.558789,-0.314733,-0.294014,-0.740905,-0.162089
2,-1.292192,-1.357708,-1.302699,-0.881298,1.873213,0.118621,0.321083,1.366159,0.826697,-0.037771,-0.188860,-0.363270,-0.354364,0.449842,-0.314733,-0.294014,-0.740905,-0.160937
3,-1.292192,-1.357708,-1.162277,-0.881298,1.873213,0.118621,0.321083,1.366159,0.763548,-0.075294,-0.140508,-0.114685,-0.354364,0.337592,-0.314733,-0.294014,-0.740905,-0.159747
4,-1.292192,-1.357708,-1.162277,-0.881298,1.873213,0.118621,0.338206,1.366159,0.037329,-0.225387,-0.237211,-0.363270,-0.354364,0.222042,-0.314733,-0.294014,-0.740905,-0.158557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10124,-1.121422,-0.709604,-0.858029,-1.246022,-0.381191,-0.282543,-0.021378,0.447895,-0.246843,-0.300433,-0.237211,-0.197546,-0.579404,0.004146,-0.314733,-0.294014,2.027820,-0.420289
10125,-1.121422,-0.828992,-0.858029,-1.414356,-0.381191,-0.282543,-0.004255,0.447895,-0.309993,-0.262910,-0.261387,-0.197546,-0.579404,0.004146,-0.314733,-0.294014,2.027820,-0.419137
10126,-0.922191,-0.828992,-0.858029,-1.414356,-0.381191,-0.282543,-0.004255,0.447895,-0.183694,-0.262910,-0.237211,-0.197546,-0.579404,0.004146,-0.314733,-0.294014,2.027820,-0.417947
10127,-0.922191,-0.828992,-0.858029,-1.414356,-0.381191,-0.282543,-0.004255,0.447895,-0.278418,-0.150341,-0.140508,-0.197546,-0.579404,0.004146,-0.314733,-0.294014,2.027820,-0.416757


In [ ]:
from minisom import MiniSom

som = MiniSom(60, 40, 12)
# get arr from dataframe

som.train(df.values, 20000)

In [ ]:
weights = som._weights

file_path = 'som_occ_l.csv'

# Save the array to a CSV file
np.savetxt(file_path, weights.reshape(-1, weights.shape[-1]), delimiter=',', fmt='%.18e')


In [ ]:
file_path = 'som_occ_l.csv'

# Load the array from the CSV file
loaded_weights = np.loadtxt(file_path, delimiter=',')

# Reshape the array to its original dimensions (60, 40, 18)
loaded_weights_reshaped = loaded_weights.reshape((300, 200, 18))

are_identical = np.array_equal(weights, loaded_weights_reshaped)
are_identical

True

In [ ]:
%pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
wine_quality = fetch_ucirepo(id=186)

# data (as pandas dataframes)
X = wine_quality.data.features
y = wine_quality.data.targets

# metadata
print(wine_quality.metadata)

# variable information
print(wine_quality.variables)

dataset = wine_quality.data.original

{'uci_id': 186, 'name': 'Wine Quality', 'repository_url': 'https://archive.ics.uci.edu/dataset/186/wine+quality', 'data_url': 'https://archive.ics.uci.edu/static/public/186/data.csv', 'abstract': 'Two datasets are included, related to red and white vinho verde wine samples, from the north of Portugal. The goal is to model wine quality based on physicochemical tests (see [Cortez et al., 2009], http://www3.dsi.uminho.pt/pcortez/wine/).', 'area': 'Business', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 4898, 'num_features': 11, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['quality'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2009, 'last_updated': 'Wed Nov 15 2023', 'dataset_doi': '10.24432/C56S3T', 'creators': ['Paulo Cortez', 'A. Cerdeira', 'F. Almeida', 'T. Matos', 'J. Reis'], 'intro_paper': {'title': 'Modeling wine preferences by data mining from physicoc

In [ ]:
import numpy as np

df2 = dataset
# Select columns to normalize
df2.drop(['quality'],1, inplace=True)
df2.drop(['color'],1, inplace=True)
columns_to_normalize = df2.select_dtypes(include=np.number).columns

# Z-score normalization
df2[columns_to_normalize] = (df2[columns_to_normalize] - df2[columns_to_normalize].mean()) / df2[columns_to_normalize].std()

# Normalize wine color
# df2['color'] = df2['color'].map({'white': 0, 'red': 1})

# Print the normalized DataFrame
df2

<ipython-input-32-6e099ed46182>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2.drop(['quality'],1, inplace=True)
<ipython-input-32-6e099ed46182>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2.drop(['color'],1, inplace=True)


,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
0,0.142462,2.188664,-2.192664,-0.744721,0.569914,-1.100055,-1.446247,1.034913,1.812950,0.193082,-0.915394
1,0.451001,3.281982,-2.192664,-0.597594,1.197883,-0.311296,-0.862402,0.701432,-0.115064,0.999502,-0.580023
2,0.451001,2.553104,-1.917405,-0.660648,1.026618,-0.874695,-1.092402,0.768128,0.258100,0.797897,-0.580023
3,3.073580,-0.362411,1.660957,-0.744721,0.541370,-0.762016,-0.986248,1.101609,-0.363840,0.327485,-0.580023
4,0.142462,2.188664,-2.192664,-0.744721,0.569914,-1.100055,-1.446247,1.034913,1.812950,0.193082,-0.915394
...,...,...,...,...,...,...,...,...,...,...,...
6492,-0.783154,-0.787590,-0.197039,-0.807775,-0.486215,-0.367636,-0.420095,-1.186069,0.320294,-0.210128,0.593772
6493,-0.474615,-0.119451,0.284664,0.537383,-0.257863,1.491582,0.924517,0.067819,-0.426034,-0.478935,-0.747709
6494,-0.551750,-0.605370,-0.885185,-0.891847,-0.429127,-0.029596,-0.083942,-0.719196,-1.421138,-0.478935,-0.915394
6495,-1.323097,-0.301671,-0.128224,-0.912866,-0.971463,-0.592996,-0.101635,-2.003097,0.755652,-1.016548,1.935253


In [ ]:
from minisom import MiniSom

som = MiniSom(200, 150, 11)
# get arr from dataframe

som.train(df2.values, 20000)

In [ ]:
weights = som._weights

file_path = 'som_wines_l.csv'

# Save the array to a CSV file
np.savetxt(file_path, weights.reshape(-1, weights.shape[-1]), delimiter=',', fmt='%.18e')

In [ ]:
file_path = 'som_wines_l.csv'

# Load the array from the CSV file
loaded_weights = np.loadtxt(file_path, delimiter=',')

# Reshape the array to its original dimensions (60, 40, 18)
loaded_weights_reshaped = loaded_weights.reshape((200, 150, 11))

are_identical = np.array_equal(weights, loaded_weights_reshaped)
are_identical

True

In [ ]:
loaded_weights = np.loadtxt('som_wines_l.csv', delimiter=',')

# Reshape the array to its original dimensions (60, 40, 18)
weights = loaded_weights.reshape((200, 150, 11))

# fetch dataset
wine_quality = fetch_ucirepo(id=186)

dataset = wine_quality.data.original
sorted_indices = dataset['quality'].argsort()
sorted_dataset = dataset.iloc[sorted_indices]

df = sorted_dataset
# Create classes from quality column
quality_classes = df['quality']
quality_classes_str = np.unique(quality_classes).astype(str)

# Select columns to normalize
df.drop(['quality'], axis=1, inplace=True)
df.drop(['color'],1, inplace=True)
columns_to_normalize = df.select_dtypes(include=np.number).columns

# Z-score normalization
df[columns_to_normalize] = (df[columns_to_normalize] - df[columns_to_normalize].mean()) / df[columns_to_normalize].std()

# Normalize wine color
# df['color'] = df['color'].map({'white': 0, 'red': 1})

from pysomvis import PySOMVis
vis = PySOMVis(weights=weights, input_data=df.values,classes_names=quality_classes_str,classes=quality_classes)


<ipython-input-36-4a01706ba37a>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['quality'], axis=1, inplace=True)
<ipython-input-36-4a01706ba37a>:20: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['color'],1, inplace=True)
<ipython-input-36-4a01706ba37a>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['color'],1, inplace=True)
<ipython-input-36-4a01706ba37a>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [ ]:
vis._mainview

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-p

Column
    [0] Column
        [0] Column
            [0] Row(margin=(5, 10), width=700)
                [0] StaticText(value='<b></b>')
                [1] Select(options=OrderedDict([('Component P...]), value=0)
                [2] Select(options=OrderedDict([('PiYG', ...]), value='jet')
                [3] Button(name='↶')
                [4] Button(name='↷')
                [5] Button(name='↔')
                [6] Button(name='↕')
                [7] Checkbox(name='interpolation')
        [1] Row
            [0] Column
                [0] HoloViews(DynamicMap, height=500, sizing_mode='fixed', width=500)
            [1] Row
                [0] Column
                    [0] Column(margin=(5, 10), name='Component Planes')
                        [0] StaticText(value='<b>Component Planes</b>')
                        [1] IntSlider(end=10, name='Component')
                    [1] Str(str)
    [1] Column
        [0] Row()

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-p